<a href="https://colab.research.google.com/github/KarelZe/adv-ml/blob/main/AML_Bonus_MarkusBilz_2327197_VerenaHollinger_2055134.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [445]:
!pip install imbalanced-learn

In [446]:
#import dependencies
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    matthews_corrcoef,
    plot_confusion_matrix,
)

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from google.colab import drive

from statsmodels.api import OLS

import seaborn as sns
sns.set_theme(style="whitegrid", palette="viridis", font_scale=1.1, color_codes=True)

import matplotlib.pyplot as plt
%matplotlib inline

# increase the number of rows displayed to fully display all (transposed) features.
pd.options.display.max_rows = 1000


# Task 1: Explain dataset and business problems in two paragraphs. (10 points)

Peer-2-Peer Lending is a decentralized approach in which investors (most oftenly private investors) lend to debtors (typically private individuals). Peer-2-peer lending platforms such as Lending Club serve as an intermediary between investors and borrowers. Due to the decentralized structure, no banks are involved. The risk for a loan is that the borrower defaults partly or entirely. Expected interest and redemption payments may then not be made. The borrowers suffer a loss. Typically, investors invest in several loans to diversify their default risk. Ergo, one loan is financed by several investors. Of key Of central interest is whether a default of a loan can be predicted and with which features. The platform could use this information to reject loans that are highly likely to default during the application process, thus avoiding defaults.

The dataset contains loan data on loans, that where distributed through the peer-2-peer lending platform Lending club. The data includes information on payment history, loan terms, borrower information and most importantly the loan status. In overview, data can be classified into hard and soft features. Some features are categorical e. g. term, while others are numerical e. g. debt-to-income-ratio or unstructured e. g. purpose.



## Loading the data

Loan data is available from kaggle. The loan data is stored unaltered in our Google Drive.

Additional income data was downloaded from the [US Census Bureau](https://data.census.gov/cedsci/table?q=S1901%3A%20INCOME%20IN%20THE%20PAST%2012%20MONTHS%20%28IN%202018%20INFLATION-ADJUSTED%20DOLLARS%29&g=0400000US01,02,04,05,06,08,09,10,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,50,51,53,54,55,56,72&tid=ACSST1Y2018.S1901&hidePreview=false). We use this information later in our analysis to compare the income of borrowers with the avg. income of the state they are located int. To be able to map US state names to their corresponding ISO-Codes we use es use the table found at [ISO.org](https://www.iso.org/obp/ui/#iso:code:3166:US) as a lookup table.

In [447]:

drive.mount("/content/drive")

data = pd.read_csv('/content/drive/My Drive/AdvancedML/accepted_2007_to_2018Q4.csv')
income_by_states = pd.read_csv('/content/drive/My Drive/AdvancedML/ACSST1Y2018_S1901_data_with_overlays_2021-06-12T020307.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Task 2: Do an explanatory analysis for this dataset. (10 points)

In [448]:
print(data.shape)

(2260701, 151)


The dataset consists of ~ 2.26 million loans and 151 features.

In [449]:
data.head().T

,0,1,2,3,4
id,68407277,68355089,68341763,66310712,68476807
member_id,NaN,NaN,NaN,NaN,NaN
loan_amnt,3600,24700,20000,35000,10400
funded_amnt,3600,24700,20000,35000,10400
funded_amnt_inv,3600,24700,20000,35000,10400
term,36 months,36 months,60 months,60 months,60 months
int_rate,13.99,11.99,10.78,14.85,22.45
installment,123.03,820.28,432.66,829.9,289.91
grade,C,C,B,C,F
sub_grade,C4,C1,B4,C5,F1


In [ ]:
data.describe(include='all').T

Not all features are known at the time, when the loan is handed out. E. g. 'hardship_dpd' is not known. Therefore, a preselection is done to those features that can be known during the application process by using a [dictionary](https://drive.google.com/file/d/1UzyKMtrzL3aEEQpeYIz_qjBWyW-lAMr0/view). We restrict our further analysis to thhose features.

Moreover, the columns `url` or `desc`, as the info is no longer available on the lending club website.

Finally, we also drop the column `id`, as it is unique to the loan, but can not be used for a default prediction.

In [ ]:
a_priori_known = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status',  'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',  'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'revol_bal_joint']

# filter columns of data frame, that can be known in advance
data = data[a_priori_known]

Next, we look at missing data.

In [ ]:
data.isna().sum()

or visually for the first 50 rows:

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.heatmap(data.head(50).isnull(), cbar=False);
plt.xlabel("feature")
plt.ylabel("row")
plt.show();

As it can be seen above, there is a high of missing data. While `dt_joint` and `acc_now_delinq` are missing more often than other features such as `title`, To further quantify this discovery, we use use a pareto plot in the next step. The missing values appear in a *completely random (MCAR)* fashion.

In [ ]:
missing_val = data.isna().sum()
missing_val.sort_values(ascending=False, inplace=True)

missing_val = missing_val[missing_val > 1]
missing_val = missing_val.to_frame()

missing_val['pareto'] = 100 *missing_val[0].cumsum() / missing_val[0].sum()
missing_val.rename(columns={0:'nan'}, inplace=True)

print(missing_val.tail())

In [ ]:
fig, axes = plt.subplots(figsize=(20,10))
ax1 = missing_val.plot(use_index=True, y='nan',  kind='bar', ax=axes)
ax2 = missing_val.plot(use_index=True, y='pareto', marker='D', color="C1", kind='line', ax=axes, secondary_y=True)
ax2.set_ylim([0,110])
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
plt.xticks(rotation=45)
plt.title("Pareto plot of missing values by column")
plt.show()

To fix the problem of columns with a high degree of missing values, we exclude columns with more than 20 % of missing values. We are aware, that by removing features a bias is introduced. However, it's common practice to remove columns above this threshold, as shown in (...).

In [ ]:
threshold = 0.80

percentage_missing_val = data.isna().sum()/len(data)

percentage_missing_val = percentage_missing_val[percentage_missing_val >= threshold]

print(percentage_missing_val)
threshold_abs = int(threshold * len(data))

data = data.dropna(axis=1, thresh=threshold_abs)

print(data.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.heatmap(data.head(50).isnull(), cbar=False);
plt.xlabel("feature")
plt.ylabel("row")
plt.show();

Looking at the Data Frame again...


Some rows are present, where the volume and many other features are null. We exclude these loans without volume from analysis.



In [ ]:
data = data[data.loan_amnt.notna()]

This leads to the following result...

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.heatmap(data.head(50).isnull(), cbar=False);
plt.xlabel("feature")
plt.ylabel("row")
plt.show();

In [ ]:
# convert to categorical
categorical = ['addr_state', 'term', 'application_type', 'initial_list_status', 'loan_status', 'grade', 'sub_grade', 'home_ownership', 'verification_status', 'purpose','emp_length']
data[categorical] = data[categorical].apply(lambda x: x.astype('category'))

# convert to string
strs = ['emp_title', 'title', 'zip_code']
data[strs] = data[strs].apply(lambda x: x.astype('string'))

# convert to datetime
dts = ['earliest_cr_line', 'issue_d', 'last_credit_pull_d']
data[dts] = data[dts].apply(lambda x: pd.to_datetime(x, format="%b-%Y",  errors='coerce'))

# convert to integer
# FIXME: Remove this piece of code, as id is dropped further above
#ints = ['id']
#data[ints] = data[ints].apply(lambda x: x.astype('int'))

print(data.dtypes)

##Number of unique classes per feature

In [ ]:
data_classes = []
for row in data.columns.tolist():
    data_classes.append({'column': row, 'count': len(data[row].unique())})
amount = pd.DataFrame(data_classes).sort_values('count')
print(amount)

The features `purpose` and `title` contain similar content. Since `purpose` only contains 14 classes and `title` contains more than 60,000 classes we keep `purpose` and drop `title`.

More over we drop `emp_title` and `zip_code`, as they contain either free-text or a high number of unique values, which is true for the later. Geographical information is also part of the variable `addr_state`, which we keep. We expect little loss in information by deleting these variables.

In [ ]:
data = data.drop(columns=['emp_title', 'zip_code', 'title'])

Next, we analyze categorical features by looking at their $k$ most frequent values.

In [ ]:
k = 10
data['term'].value_counts().head(k)

In [ ]:
data['addr_state'].value_counts().head(k)

In [ ]:
data['application_type'].value_counts().head(k)

In [ ]:
data['initial_list_status'].value_counts().head(k)

In [ ]:
data['loan_status'].value_counts().head(k)

In [ ]:
data['grade'].value_counts().head(k)

In [ ]:
data['sub_grade'].value_counts().head(k)

In [ ]:
data['home_ownership'].value_counts().head(k)

In [ ]:
data['verification_status'].value_counts().head(k)

In [ ]:
data['purpose'].value_counts().head(k)

In [ ]:
data['emp_length'].value_counts().head(k)

##Analyze loan status

In [ ]:
data['loan_status'].value_counts()

In [ ]:
loan_status_absolute = data['loan_status'].value_counts()
loan_status_absolute = loan_status_absolute.to_frame()

fig, ax = plt.subplots(figsize=(20,10))
plt.ticklabel_format(style='plain')
sns.barplot(x='loan_status', y=loan_status_absolute.index,  data=loan_status_absolute, ax=ax);

In [ ]:
default_categories = ['Default', 'Charged Off', 'Does not meet the credit policy. Status:Charged Off']
data['TARGET'] = data['loan_status'].isin(default_categories)

data['TARGET'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
plt.ticklabel_format(style='plain')
g = sns.countplot(x='TARGET', hue='loan_status', data=data);
g.legend(loc='upper right', bbox_to_anchor=(1, 1), ncol=1);
plt.show();

## default status by debt-to-income-ratio
- For calculation of debt-to-income-ratio see [here.](https://www.lendingclub.com/loans/resource-center/calculating-debt-to-income)

**expectation**
- One would expect more defaults with a higher debt-to-income ratio, but no clear trend can be derived from data.

In [ ]:
g = sns.displot(data, x='dti', hue='TARGET', multiple='fill', legend=False, height=10, aspect=20/10, alpha=1, bins=1000)
plt.title("default status by debt to income ratio in relative numbers")
plt.xlim(0, 100)
plt.legend(title='Defaulted', loc='upper right', labels=['Default', 'Non-Default'])
plt.show(g);

## default status by term structure and amount

In [ ]:
fig = plt.figure(figsize=(20,10))
sns.violinplot(x="TARGET",y="loan_amnt",data=data, hue="term", split=True)
plt.title("default by term structure and amount")
plt.xlabel("defaulted")
plt.ylabel("Loan amount");

## default status by grade

**expectation**
- Loans with a lower rating inherit a higher risk of default.

In [ ]:
g = sns.displot(data, x='grade', hue='TARGET', multiple='stack', legend=False, height=10, aspect=20/10, alpha=1);
plt.title("default status by grade")
plt.legend(title='Defaulted', loc='upper right', labels=['Default', 'Fully paid'])
plt.show(g);

In [ ]:
g = sns.displot(data, x='grade', hue='TARGET', multiple='fill', legend=False, height=10, aspect=1, alpha=1)
plt.title("default status by grade in relative numbers")
plt.legend(title='Defaulted', loc='upper right', labels=['Default', 'Non-Default'])
plt.show(g);

## default status by grade and interest rate

In [ ]:
fig = plt.figure(figsize=(20,10))
sns.violinplot(x="TARGET",y="int_rate",data=data, hue="grade",)
plt.title("interest rate by grade and default status")
plt.xlabel("defaulted")
plt.ylabel("Interest Rate");

# Default status by application type

**expectation**
- Joint applications inherit lower risk of default.

In [ ]:
fig = plt.figure(figsize=(18,8))
sns.violinplot(x="TARGET",y="loan_amnt",data=data, hue="application_type", split=True)
plt.title("default by amount and application type")
plt.xlabel("defaulted")
plt.legend(loc='upper right', bbox_to_anchor=(1, 1), ncol=1, labels=['Individual', 'Joint Application']);
plt.ylabel("Loan Amount");

## Default status by working experience

**expectation**
- The longer the working experience, the smaller the probability of default.

In [ ]:
fig = plt.figure(figsize=(20,10))
sns.countplot(x='emp_length', hue='TARGET', data=data)
plt.xlabel('Employee Working Experience')
plt.legend(title='Defaulted', loc='upper right', labels=['Default', 'Non-Default'])
plt.show()

In [ ]:
#fig = plt.figure(figsize=(20,10))

#tickvalues = range(0,len(x))
#g = sns.displot(data, x='emp_length', hue='TARGET', multiple='fill', legend=False, height=10, aspect=20/10, alpha=1, xticks=[])
#plt.title("default status by working experience in relative numbers")
#plt.legend(title='Defaulted', loc='upper right', labels=['Default', 'Non-Default'])
#plt.xlabel("Employee Working Experience")
#plt.ylabel("density")
#plt.show();

## Default status by state

**expectation**
- None.

In [ ]:
fig, ax = plt.subplots(figsize=(10,20))
plt.ticklabel_format(style='plain')
g = sns.countplot(y='addr_state', hue='TARGET', data=data, ax=ax);
g.legend(loc='upper right', bbox_to_anchor=(1, 1), ncol=1, labels=['Default', 'Fully paid']);
plt.ylabel("state")
plt.xlabel("count")
plt.show();

In [ ]:
g = sns.displot(data, y='addr_state', hue='TARGET', multiple='fill', legend=False, height=20, aspect=10/20, alpha=1)
plt.title("default status by state in relative numbers")
plt.legend(title='Defaulted', loc='upper right', labels=['Default', 'Non-Default'])
plt.ylabel("state")
plt.xlabel("density")
plt.show(g);

## default status by purpose

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plt.ticklabel_format(style='plain')
g = sns.countplot(y='loan_status', hue='TARGET', data=data, ax=ax);
g.legend(loc='upper right', bbox_to_anchor=(1, 1), ncol=1, labels=['Default', 'Fully paid']);
plt.ylabel("loan status")
plt.show();

## Interest rate by grade

In [ ]:
plt.figure(figsize=(12,10))
g = sns.boxplot(data=data, x='grade', y='int_rate')
plt.title("interest rate by grade")
plt.show(g);

## default status by interest rate

In [ ]:
plt.figure(figsize=(20,10))
bins = np.linspace(5, 33, 33)
g = sns.histplot(data, x='int_rate', hue='TARGET', multiple='dodge', legend=False, alpha=1, bins=bins);
plt.title("default status by interest rate")
plt.legend(title='Defaulted', loc='upper right', labels=['Default', 'Fully paid'])
plt.xlabel("interest rate")
plt.ylabel("count")
plt.show(g);

In [ ]:
g = sns.displot(data, x='int_rate', hue='TARGET', multiple='fill', legend=False, height=10, bins = 50, aspect=20/10, alpha=1)
plt.title("default status by interest rate in relative numbers")
plt.legend(title='Defaulted', loc='upper right', labels=['Default', 'Non-Default'])
plt.xlabel("itnerest rate in %")
plt.ylabel("density")
plt.show();

## Analyze income


In [ ]:
income_by_states = income_by_states[['NAME', 'S1901_C01_013E']]
income_by_states.rename(columns={'NAME':'state','S1901_C01_013E': 'income'}, inplace=True)

income_by_states.head(60)

In [ ]:
state_map = pd.read_csv("/content/drive/My Drive/AdvancedML/states_mapping.csv", sep=';')
state_map.head(50)

In [ ]:
# remove comments from df
income_by_states.drop(income_by_states.index[[0]], inplace=True)

# add iso codes and make subselection
income_by_states = income_by_states.merge(state_map,how='left',left_on='state', right_on='subdivision_name')
income_by_states = income_by_states[['iso_code', 'income']]

income_by_states.head(60)

In [ ]:
lut_income = dict(income_by_states.values)
data["addr_state_avg_income"] = data["addr_state"].replace(lut_income)
data["addr_state_avg_income"] = data["addr_state_avg_income"].apply(lambda x: pd.to_numeric(x,errors="coerce")) 
data.head().T

In [ ]:
fig = plt.figure(figsize=(20,10))
bins = np.linspace(0, 300000, 60)
sns.histplot(data= data, x='annual_inc', bins=bins, kde=False, alpha=1)
plt.title("Distribution of annual income")
plt.xlim(0, 300000)
plt.xlabel("annual income")
plt.ylabel("count")
plt.show()

## Annual Income and Loan Amount of defaulted loans

In [ ]:
fig = plt.figure(figsize=(20,20))
sel_annual_inc_defaulted= data[(data.annual_inc <= 300000) & (data.TARGET==1)]
sns.jointplot(data=sel_annual_inc_defaulted, x="annual_inc", y="loan_amnt", kind="hex", height=12)
plt.xlabel("annual income")
plt.ylabel("loan amount")
plt.title("Distribution of defaulted loans by annual income and loan amount")
plt.show()

##Analyze effect of joint/ individual application

In [ ]:
data['application_type'].value_counts()

In [ ]:
g = sns.displot(data, x='application_type', hue='TARGET', multiple='stack', legend=False, alpha=1, height = 10, aspect=10/10)
plt.xlabel("Application Type")
plt.title("default status by application type")
plt.legend(title='Defaulted', loc='upper right', labels=['Default', 'Non-Default'])

plt.show(g);

In [ ]:
# enable again for final file
# data_jointapp = data.loc[data['application_type'] == 'Joint App']
# data_indidualapp = data.loc[data['application_type'] == 'Individual']

In [ ]:
# die Methode ist irgendwo rauskopiert, wenn wir sie drin lassen wollen muss ich sie noch ein bisschen was abändern
def null_values(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [ ]:
# FIXME: Enable again for final file
# miss_values = null_values(data_jointapp)
# miss_values.head(20)

In [ ]:
# miss_values = null_values(data_indidualapp)
# miss_values.head(20)

In [ ]:
# miss_values = null_values(data)
# miss_values.head()


## Data Preprocessing

As the amount of missing values, that remain in the dataset, is relatively small, we fill missing values with the modus of each feature.

In [ ]:
data.fillna(data.mode().iloc[0], inplace = True)

This leads to the following result...

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.heatmap(data.head(50).isnull(), cbar=False);
plt.xlabel("feature")
plt.ylabel("row")
plt.show();

As seen above the dataset contains 2,260,701 entries. This sheer amount of data is hard is hard to process with the tools and processing ressources, that we have to our disposal. Therefore, we apply stratified sampling first on the dataset to create a subset with a similiar distribution to the original data. The resulting dataset will be 20 % of the original data set.

In [ ]:
# Remove target from features
features = data.drop('TARGET', axis=1)
# create a 20 % random subselection
# TODO: Decreased to 5 % for faster training. Change to 20 % later.
X_stratified, _, y_stratified, _ = train_test_split(features,data.TARGET, test_size=0.95, random_state=42, stratify=data.TARGET)

Next, we split the stratified data into two parts. Namely, training data and test data. The training data will be used to train the models and for in-sample testing. Whereas the test data is used for out-of-sample testing. This time we use a 70 % / 30 % split.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_stratified, y_stratified, stratify=y_stratified, test_size=0.3, random_state=42)

In [ ]:
print(X_train.dtypes)

As seen during data exploration the data set is highly imbalanced. Non-default loans are much more frequent than defaulted loans. We use a variant of 'SMOTE' ([see](https://arxiv.org/pdf/1106.1813.pdf) and [see](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html)) to upsample the defaulted class, which is relatively rare in the raw data. This sampling is only applied on the training data.

In [ ]:
# FIXME: research proper ways of handling strings and timestamps 
# or reposition in jupyter notebook and use standard implementation.
# Currently disabled to improve performance while development

# categorial = X_train.select_dtypes(include=['category','string', 'datetime64[ns]'])
# categorial_mask = np.isin(X_train.columns, categorial.columns)

# smote = SMOTENC(random_state=42, k_neighbors=2**7, categorical_features=categorial_mask)
# X_train, y_train = smote.fit_resample(X_train, y_train)

Having `datetime64[ns]` dtypes e. g. for `ìssue_d` is desireable for visualization. However, using these datetypes in our models causes difficulties, as the datetime object have to be converted to some numeric value. We only keep the 'year' component.

In [ ]:
# See discussion here  https://stats.stackexchange.com/a/311498
# FIXME: We could also use year info and month info and keep themn in two variables

# apply to Test and Training set
datetime_columns = data.select_dtypes(include=['datetime64[ns]']).columns
X_train[datetime_columns] = X_train[datetime_columns].apply(lambda x: x.dt.year)
X_test[datetime_columns] = X_test[datetime_columns].apply(lambda x: x.dt.year)

print(X_train.dtypes)

While Variables of type 'Categorical' are great for visualization, they also require further processing to be used by machine learning models. We apply some Label-encoding.

In [ ]:
# apply to test and training set
cat_columns = data.select_dtypes(include=['category']).columns
X_train[cat_columns] = X_train[cat_columns].apply(lambda x: x.cat.codes)
X_test[cat_columns] = X_test[cat_columns].apply(lambda x: x.cat.codes)

print(X_train.dtypes)

In [ ]:
# Implementation Verena One-Hot

#data_cat = data.select_dtypes(include=['category'])
#data_cat_columns = data_cat.columns
#data_cat_columns
#data = data.drop(columns=data_cat_columns)
#data_to_encode = data_cat.drop(columns=['loan_status'])
#data_encoded = pd.get_dummies(data_to_encode)
#data_encoded


Since the features have different units, Scaling and Standardization of the numeric features are necessary. And since a Support Vector Machine required data with values between 0 and 1 the Standardization is conducted first.

In [ ]:
# Implementation Verena Min-Max-Scaling + Standard-Scaler

#min_max = MinMaxScaler()
#data_scaled = pd.DataFrame(min_max.fit_transform(data[data_num_columns]), columns = data_num_columns)
#data_scaled.describe()

#data_num = data.select_dtypes(include=[np.float64, np.float32])
#data_num_columns = data_num.columns


# TODO: Remove the following code
#data = data.drop(columns=data_num_columns)

#standardizer = StandardScaler()
#data_standardized = pd.DataFrame(standardizer.fit_transform(data_num), columns = data_num_columns)
#data_standardized.describe()

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

If there are only two classes, they have a correlation of 1, so one class can be dropped without loosing any information

In [ ]:
#data_encoded = data_encoded.drop(columns=['term_ 60 months', 'initial_list_status_w', 'application_type_Joint App'])
#data_encoded

In [ ]:
#target = data[['TARGET']]
#data = data.drop(columns=['TARGET'])
#data = pd.concat([data, data_scaled, data_encoded], axis=1)
#data[['Target']] = target
#data

In [ ]:
# TODO: Remove the following code
#data_to_encode = data[['term', 'verification_status','application_type']]
#enc = OneHotEncoder()
#df_num = enc.fit_transform(data_to_encode).toarray()
#df_num = pd.DataFrame(df_num)
#df_num

To identify similar features and relevant features (high correlation with target feature) a correlation analysis is conducted

In [ ]:
#correlation with target feature
#corr = data.corr()['TARGET'].sort_values()
#print('Most Positive Correlations:\n',corr.tail(15))
#print('\nMost Negative Correlations:\n',corr.head(15))

## Task 3: Use LASSO for variable selection and explain results. (15 points)

In Task 3 we use a LASSO model for variable selection. First we run a grid search to find a suitable value for the penalty weight $\alpha$. We use the metric Area under the curve (AUC) is the criteria for model selection.

In [ ]:
# TODO: Not yet implemented.

Next we look at variables, that....

## Task 4: Use logistic regression (in-sample) for default prediction. Interpret your results for logistic regression. (20 points)

In [ ]:
# TODO: Not yet implemented.

## Task 5: Compare logistic regression, classification tree, random forest, bagging, support vector machine (with two kernel functions), and neural network for default prediction based on the out-of-sample performance measures (at least three different measures). (35 points)

### Preliminary

In [ ]:
def print_results(model):

    y_pred_model = model.predict(X_test)

    # TODO: Use Labels from Label Encoder
    print(classification_report(y_test, y_pred_model))
    
    plot_confusion_matrix(model, X_test, y_test, display_labels=['Non-Default', 'Default'])
    plt.show()

    print(f"Accuracy: {accuracy_score(y_test, y_pred_model):.2f}")
    print(f"Params: {model.get_params()}")

### Decision Tree

In [ ]:
# max_depth = [2, 6, 10, 14...]
min_depth = 2
step_size = 4
step_count = 2**4
max_depth = min_depth + np.arange(step_count) * step_size

# perform grid search
param_grid = {'criterion': ['gini', 'entropy'], 'max_depth': max_depth}
grid_dt = GridSearchCV(DecisionTreeClassifier(), param_grid, refit=True, verbose=0)
grid_dt.fit(X_train, y_train)

# choose best model
grid_dt_optimized = grid_dt.best_estimator_

# print confusion 
print_results(grid_dt_optimized)

### Random Forest

In [ ]:
# [2, 6, 10, 14...]
max_depth = np.arange(2, 22, 4)
# [5, 10, 25, 50, 100, 150]
n_estimators = [5, 10, 25, 50, 100, 150]

# perform grid search
param_grid = {'criterion': ['gini', 'entropy'], 'max_depth': max_depth, 'n_estimators': n_estimators, 'max_features': ['auto', 'sqrt', 'log2']}
grid_rf = GridSearchCV(RandomForestClassifier(), param_grid, refit=True, verbose=0)
grid_rf.fit(X_train, y_train)

# choose best model
grid_rf_optimized = grid_rf.best_estimator_

# print confusion 
print_results(grid_rf_optimized)

### Bagging

In [ ]:
# TODO: Not yet implemented...

### Support Vector Machine

Next, we look at SVM with two different kernels. Namely, a rfb kernel and linear kernel.

In [ ]:
min_c = 0.1
step_size = 0.1
step_count = 15

C = min_c + np.arange(step_count) * step_size

param_grid = {'C': C, 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf', 'linear']}
grid_svm = GridSearchCV(SVC(), param_grid, refit=True, verbose=0)
grid_svm.fit(X_train, y_train.values.ravel())

# choose best model
grid_svm_optimized = grid_svm.best_estimator_

# print confusion 
print_results(grid_svm_optimized)

### Neural Network 

In [ ]:
param_grid = {'activation': ['logistic', 'relu'], 'learning_rate': ['constant', 'invscaling'], 'hidden_layer_sizes': [(100, 1), (100, 2)], 'early_stopping': True}
grid_nn = GridSearchCV(MLPClassifier(), param_grid, refit=True, verbose=0)
grid_nn.fit(X_train, y_train.values.ravel())

# choose best model
grid_nn_optimized = grid_nn.best_estimator_

# print confusion 
print_results(grid_nn_optimized)

### Voting Classifier

In [ ]:
# TODO: Optional. Combine the best Classifiers e. g. SVM and RF
model_voting = VotingClassifier(estimators=[('SVM', grid_svm_optimized), ('Random Forest', model_rf_optimized)],
                                   voting='hard')
model_voting.fit(X_train, y_train.values.ravel())

# print confusion matrix
print_result(model_voting)

Bringing it all together....

In [ ]:
# TODO: Performance Measures: Area under the Curve, ROC. Could be easily done with: https://www.scikit-yb.org/en/latest/api/classifier/rocauc.html

## Task 6: Which model do you suggest to the peer-to-peer company? Why? (10 points)

In [ ]:
# TODO: Not yet implemented...